In [2]:
# Read in libraries
import os
import csv
import pandas as pd

# Set the directory path where your files are located
folder_path = "C:\\Users\\johnh\\OneDrive\\Documents\\GitHub\\water-supply-forecast\\assets\\data\\teleconnections"

In [39]:
# Basic cleaning for MJO dataset
df_mjo = pd.read_table(os.path.join(folder_path,"mjo.txt"),delim_whitespace=True, skiprows=1)
df_mjo = df_mjo.iloc[1:]
df_mjo.columns = df_mjo.columns.str.strip()

df_mjo['year'] = df_mjo['PENTAD'].astype(str).str[:4].astype(int)
df_mjo['month'] = df_mjo['PENTAD'].astype(str).str[4:6].astype(int)
df_mjo['day'] = df_mjo['PENTAD'].astype(str).str[6:8].astype(int)

In [40]:
print(df_mjo)

        PENTAD    20E    70E    80E   100E   120E   140E   160E   120W    40W  \
1     19780108  -0.99  -0.27   0.82   1.39   1.34   1.10   0.57  -0.26  -1.18   
2     19780113  -1.10  -1.17  -0.65   0.33   0.60   1.03   1.18   0.96   0.14   
3     19780118   1.68   0.88  -0.67  -1.78  -1.70  -1.73  -1.25  -0.20   1.25   
4     19780123   2.29   2.45   1.33  -0.69  -1.12  -2.12  -2.50  -2.07  -0.37   
5     19780128   1.25   2.22   2.30   0.87   0.35  -0.94  -1.91  -2.44  -1.72   
...        ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
3353  20231209  *****  *****  *****  *****  *****  *****  *****  *****  *****   
3354  20231214  *****  *****  *****  *****  *****  *****  *****  *****  *****   
3355  20231219  *****  *****  *****  *****  *****  *****  *****  *****  *****   
3356  20231224  *****  *****  *****  *****  *****  *****  *****  *****  *****   
3357  20231229  *****  *****  *****  *****  *****  *****  *****  *****  *****   

        10W  
1     -1.36  

In [ ]:
    # Open the file for reading
    with open(folder_path, "r") as file:
        # Read the entire file contents
        file_contents = file.read()
        print("File Contents:")
        print(file_contents)

    except FileNotFoundError:
        print(f"The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")